In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import optuna
import warnings
warnings.filterwarnings('ignore')
# Fijar semillas para reproducibilidad
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

In [28]:
df_train = pd.read_parquet('../../Datos/df_train.parquet')
df_test  = pd.read_parquet('../../Datos/df_test.parquet')

In [29]:
# Crear columna semana con dos dígitos
for df in [df_train, df_test]:
    df['semana_str'] = df['semana'].astype(str).str.zfill(2)
    df['date'] = pd.to_datetime(
        df['anio'].astype(str) + df['semana_str'] + '1',
        format='%G%V%u'  # %G: año ISO, %V: semana ISO, %u: día de la semana (1=Lun)
    )

La clase **DengueDataset** transforma eficazmente los datos crudos de casos de dengue, agrupados por ubicación, en un formato adecuado para modelos de predicción de secuencias. Crea secuencias superpuestas de casos de dengue pasados (x) para predecir el siguiente caso de dengue (y), asegurando que los datos estén correctamente estructurados y tipados para su uso con modelos de PyTorch.

In [30]:
import torch # Importa la librería PyTorch, fundamental para tensores y redes neuronales
from torch.utils.data import Dataset # Importa la clase base Dataset de PyTorch, que necesitamos para crear nuestro dataset personalizado

class DengueDataset(Dataset):
    """
    Clase personalizada para manejar los datos de dengue, preparándolos para modelos de series de tiempo en PyTorch.
    Hereda de torch.utils.data.Dataset, lo que permite que PyTorch's DataLoader la use fácilmente.
    """
    def __init__(self, df, seq_len):
        """
        Constructor de la clase DengueDataset.

        Args:
            df (pd.DataFrame): DataFrame de Pandas con los datos de dengue.
                              Debe contener al menos las columnas 'id_bar', 'date' y 'dengue'.
            seq_len (int): Longitud de la secuencia de entrada (número de pasos de tiempo pasados
                           a usar para la predicción).
        """
        self.seq_len = seq_len # Almacena la longitud de la secuencia para uso posterior
        self.samples = []      # Lista donde almacenaremos los pares (secuencia de entrada, valor objetivo)

        # Agrupar por 'id_bar' (identificador de barrio/área) para procesar cada serie de tiempo individualmente.
        # Esto es crucial porque cada barrio es una serie de tiempo independiente.
        for bar, grp in df.groupby('id_bar'):
            # Ordenar los valores de dengue por fecha para asegurar el orden cronológico.
            # Convertimos la columna 'dengue' a un array de NumPy para facilitar el slicing.
            vals = grp.sort_values('date')['dengue'].values

            # Solo creamos muestras si la serie de tiempo es lo suficientemente larga.
            # Necesitamos al menos 'seq_len' valores para la entrada (x) y 1 valor para el objetivo (y).
            if len(vals) >= seq_len + 1:
                # Iteramos para crear "ventanas" deslizantes de datos.
                # 'i' será el índice de inicio de cada secuencia de entrada (x).
                for i in range(len(vals) - seq_len):
                    # Extraemos la secuencia de entrada (x): 'seq_len' valores desde el índice 'i'.
                    x = vals[i: i + seq_len]
                    # Extraemos el valor objetivo (y): el valor inmediatamente después de la secuencia x.
                    y = vals[i + seq_len]
                    # Añadimos el par (x, y) a nuestra lista de muestras.
                    self.samples.append((x, y))

    def __len__(self):
        """
        Método requerido por torch.utils.data.Dataset.
        Devuelve el número total de muestras (pares de entrada-salida) en el dataset.
        """
        return len(self.samples)

    def __getitem__(self, idx):
        """
        Método requerido por torch.utils.data.Dataset.
        Devuelve una muestra específica (x, y) dado un índice.

        Args:
            idx (int): El índice de la muestra a recuperar.

        Returns:
            tuple: Una tupla que contiene:
                   - x (torch.Tensor): La secuencia de entrada, con forma [seq_len, 1].
                   - y (torch.Tensor): El valor objetivo.
        """
        x, y = self.samples[idx] # Obtiene la secuencia de entrada (x) y el valor objetivo (y) de la lista.

        # Convierte x a un tensor de PyTorch con tipo float32.
        # .unsqueeze(-1) añade una dimensión al final, cambiando la forma de [seq_len] a [seq_len, 1].
        # Esto es común para modelos RNN/LSTM que esperan características en la última dimensión.
        x_tensor = torch.tensor(x, dtype=torch.float32).unsqueeze(-1)

        # Convierte y a un tensor de PyTorch con tipo float32.
        # y es un valor escalar, por lo que no necesita un unsqueeze.
        y_tensor = torch.tensor(y, dtype=torch.float32)

        return x_tensor, y_tensor # Devuelve los tensores de entrada y salida

El modelo **GRUForecast** está diseñado para tomar una secuencia de datos de dengue (x, por ejemplo, los casos de las últimas 7 semanas) y predecir el número de casos de dengue para la siguiente semana (y). La GRU es experta en encontrar patrones y dependencias dentro de esa secuencia temporal.

In [31]:
import torch.nn as nn # Importa el módulo 'nn' de PyTorch, que contiene las capas de redes neuronales

class GRUForecast(nn.Module):
    """
    Define un modelo de pronóstico de series de tiempo utilizando una red GRU.
    Hereda de nn.Module, la clase base para todos los módulos de redes neuronales en PyTorch.
    """
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        """
        Constructor del modelo GRUForecast.

        Args:
            input_size (int): El número de características en cada paso de tiempo de la secuencia de entrada.
                              En nuestro caso de dengue, es 1 (el número de casos de dengue).
            hidden_size (int): El número de características en el estado oculto (o celdas) de la GRU.
                               Determina la capacidad de la GRU para aprender patrones complejos.
            num_layers (int): El número de capas GRU apiladas. Más capas pueden aprender
                              representaciones de nivel superior, pero aumentan la complejidad.
            dropout (float): La probabilidad de aplicar Dropout en las capas GRU (excepto la última).
                             Ayuda a prevenir el sobreajuste.
        """
        super().__init__() # Llama al constructor de la clase padre (nn.Module). ¡Siempre necesario!

        # Define la capa GRU (Gated Recurrent Unit).
        self.gru = nn.GRU(
            input_size=input_size,    # Tamaño de las características de entrada en cada paso de tiempo.
            hidden_size=hidden_size,  # Tamaño del estado oculto de la GRU.
            num_layers=num_layers,    # Número de capas GRU apiladas.
            dropout=dropout,          # Tasa de dropout aplicada entre las capas GRU.
            batch_first=True          # Importante: Indica que la dimensión de lote (batch) es la primera.
                                      # Las entradas esperadas serán de la forma [batch, seq_len, features].
                                      # Si fuera False, sería [seq_len, batch, features].
        )

        # Define la capa de salida.
        # Es una capa completamente conectada (lineal) que mapea la salida del estado oculto
        # de la GRU (hidden_size) a una única predicción (1).
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        """
        Define el pase hacia adelante (forward pass) del modelo.
        Describe cómo los datos fluyen a través de las capas del modelo.

        Args:
            x (torch.Tensor): El tensor de entrada al modelo, con forma [batch_size, sequence_length, input_size].
                              Para nuestro caso de dengue, sería [batch_size, seq_len, 1].

        Returns:
            torch.Tensor: El tensor de predicciones de salida, con forma [batch_size].
                          Cada valor es la predicción de casos de dengue para el siguiente paso de tiempo.
        """
        # x: El tensor de entrada. Su forma es [batch_size, seq_len, features].
        # La GRU procesa la secuencia de entrada.
        # 'out' contendrá la salida del estado oculto para CADA paso de tiempo de la secuencia.
        # 'hidden' contendrá el estado oculto final después de procesar toda la secuencia.
        out, _ = self.gru(x) # out: [batch_size, seq_len, hidden_size]. El segundo valor '_' se ignora (estado oculto final).

        # Tomamos la salida del ÚLTIMO paso de tiempo de la secuencia.
        # Esta es la salida de la GRU que encapsula la información de toda la secuencia.
        # out[:, -1, :] selecciona todos los elementos del lote (:) en el último paso de tiempo (-1)
        # y todas las características del estado oculto (:).
        last = out[:, -1, :] # last: [batch_size, hidden_size]

        # Pasamos la salida del último paso de tiempo a través de la capa lineal (completamente conectada).
        # Esto reduce el 'hidden_size' a nuestro valor de predicción único (1).
        # .squeeze() elimina dimensiones de tamaño 1. Si la salida fuera [batch_size, 1],
        # .squeeze() la convierte a [batch_size], que es el formato deseado para predicciones escalares.
        return self.fc(last).squeeze() # Salida final: [batch_size]

In [32]:
# Función objetivo para la optimización de hiperparámetros con Optuna
def objective(trial):
    """
    Función objetivo para la optimización de hiperparámetros con Optuna.
    Optuna llamará a esta función repetidamente, sugiriendo diferentes hiperparámetros
    en cada 'trial' (intento), y esta función devolverá la métrica de rendimiento
    (en este caso, la pérdida en el conjunto de validación) para ese conjunto de hiperparámetros.

    Args:
        trial (optuna.trial.Trial): Objeto Trial proporcionado por Optuna, que permite
                                    sugerir hiperparámetros y gestiona el estado del intento.

    Returns:
        float: El valor de la métrica objetivo (la pérdida promedio de MSE en el conjunto de validación).
               Optuna intentará minimizar este valor.
    """
    # --- 1. Sugerencias de Hiperparámetros con Optuna ---
    # Optuna explora diferentes valores para estos parámetros en cada 'trial'.

    # Longitud de la secuencia de entrada para la GRU.
    # trial.suggest_int: Sugiere un entero en un rango definido, con un paso específico.
    seq_len     = trial.suggest_int('seq_len', 4, 52, step=4) # Rango de 4 a 52, en incrementos de 4.

    # Tamaño de la capa oculta de la GRU.
    # trial.suggest_categorical: Sugiere un valor de una lista predefinida de opciones.
    hidden_size = trial.suggest_categorical('hidden_size', [16, 32, 64, 128]) # Opciones discretas.

    # Número de capas GRU apiladas.
    num_layers  = trial.suggest_int('num_layers', 1, 2) # Puede ser 1 o 2 capas.

    # Tasa de dropout para regularización.
    # trial.suggest_float: Sugiere un número flotante en un rango continuo.
    dropout     = trial.suggest_float('dropout', 0.0, 0.5) # Rango de 0.0 a 0.5.

    # Tasa de aprendizaje para el optimizador Adam.
    # trial.suggest_loguniform: Sugiere un flotante en un rango logarítmico (útil para tasas de aprendizaje).
    lr          = trial.suggest_loguniform('lr', 1e-4, 1e-2) # Rango logarítmico de 0.0001 a 0.01.

    # Tamaño del lote para el entrenamiento.
    batch_size  = trial.suggest_categorical('batch_size', [16, 32, 64]) # Opciones discretas.

    # --- 2. Preparar Datos y DataLoaders ---
    # Se utiliza el dataset y los loaders con los hiperparámetros sugeridos.

    # Crea una instancia de nuestro DengueDataset con el 'df_train' y el 'seq_len' sugerido.
    dataset = DengueDataset(df_train, seq_len)

    # Divide el dataset en conjuntos de entrenamiento y validación.
    # n_train: 80% del dataset para entrenamiento.
    n_train = int(len(dataset) * 0.8)
    # n_val: El 20% restante para validación.
    n_val   = len(dataset) - n_train
    # random_split divide el dataset de forma aleatoria en los tamaños especificados.
    train_ds, val_ds = torch.utils.data.random_split(dataset, [n_train, n_val])

    # Crea DataLoaders para iterar sobre los datos en lotes.
    # train_loader: Para el entrenamiento, con 'batch_size' sugerido y datos mezclados (shuffle=True).
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    # val_loader: Para la validación, con 'batch_size' sugerido (no es necesario mezclar para validación).
    val_loader   = DataLoader(val_ds, batch_size=batch_size)

    # --- 3. Modelo, Optimizador y Criterio de Pérdida ---
    # Inicializa los componentes del modelo con los hiperparámetros sugeridos.

    # Instancia el modelo GRUForecast. input_size es 1 porque tenemos una sola característica (dengue cases).
    model     = GRUForecast(1, hidden_size, num_layers, dropout)
    # Define el optimizador (Adam es una buena opción para redes neuronales) con la tasa de aprendizaje sugerida.
    optimizer = optim.Adam(model.parameters(), lr=lr)
    # Define la función de pérdida (Mean Squared Error - MSE), común para problemas de regresión.
    criterion = nn.MSELoss()

    # --- 4. Entrenamiento Rápido del Modelo ---
    # Se realiza un entrenamiento simplificado con un número fijo de épocas (ej. 10)
    # para evaluar el rendimiento de esta combinación de hiperparámetros.
    # En un entrenamiento completo, esto sería más largo.
    for epoch in range(10): # Iteramos por un número fijo de épocas de entrenamiento
        model.train() # Pone el modelo en modo de entrenamiento (activa dropout, etc.)
        for x_batch, y_batch in train_loader: # Itera sobre los lotes del conjunto de entrenamiento
            optimizer.zero_grad() # Reinicia los gradientes acumulados de la iteración anterior
            y_pred = model(x_batch) # Realiza un pase hacia adelante: predice 'y' a partir de 'x'
            loss = criterion(y_pred, y_batch) # Calcula la pérdida entre las predicciones y los valores reales
            loss.backward() # Calcula los gradientes de la pérdida con respecto a los parámetros del modelo
            optimizer.step() # Actualiza los pesos del modelo usando los gradientes y la tasa de aprendizaje

    # --- 5. Validación del Modelo ---
    # Evalúa el rendimiento del modelo entrenado en el conjunto de validación.

    model.eval() # Pone el modelo en modo de evaluación (desactiva dropout, etc.)
    losses = []  # Lista para almacenar las pérdidas de cada lote de validación
    with torch.no_grad(): # Desactiva el cálculo de gradientes para ahorrar memoria y acelerar
                          # la inferencia, ya que no estamos entrenando en esta fase.
        for x_batch, y_batch in val_loader: # Itera sobre los lotes del conjunto de validación
            y_pred = model(x_batch) # Realiza predicciones
            # Calcula la pérdida para el lote actual y la añade a la lista.
            # .item() se usa para obtener el valor escalar de un tensor de PyTorch.
            losses.append(criterion(y_pred, y_batch).item())

    # --- 6. Devolver la Métrica Objetivo ---
    # El valor promedio de las pérdidas en el conjunto de validación.
    # Optuna intentará minimizar este valor para encontrar los mejores hiperparámetros.
    return np.mean(losses) # Retorna el promedio de las pérdidas de validación

In [33]:
# Ejecutar la búsqueda
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=60, show_progress_bar=True)

print('Mejores parámetros:', study.best_params)

[I 2025-06-29 13:29:49,598] A new study created in memory with name: no-name-75da7a11-1dd2-46f7-8378-3bb43caced11


  0%|          | 0/60 [00:00<?, ?it/s]

[I 2025-06-29 13:30:05,760] Trial 0 finished with value: 4.3589935952966865 and parameters: {'seq_len': 24, 'hidden_size': 128, 'num_layers': 2, 'dropout': 0.2765393101316985, 'lr': 0.0004984585009470663, 'batch_size': 64}. Best is trial 0 with value: 4.3589935952966865.
[I 2025-06-29 13:30:18,989] Trial 1 finished with value: 3.9438720616427334 and parameters: {'seq_len': 20, 'hidden_size': 128, 'num_layers': 2, 'dropout': 0.37100886065788374, 'lr': 0.009175082401699354, 'batch_size': 64}. Best is trial 1 with value: 3.9438720616427334.
[I 2025-06-29 13:30:21,125] Trial 2 finished with value: 5.273074388504028 and parameters: {'seq_len': 24, 'hidden_size': 16, 'num_layers': 1, 'dropout': 0.003644705983679941, 'lr': 0.0015605076916763833, 'batch_size': 64}. Best is trial 1 with value: 3.9438720616427334.
[I 2025-06-29 13:30:29,685] Trial 3 finished with value: 5.21680723325066 and parameters: {'seq_len': 4, 'hidden_size': 64, 'num_layers': 2, 'dropout': 0.18255394290364552, 'lr': 0.004

In [34]:
best = study.best_params
# Preparar dataset completo
dataset = DengueDataset(df_train, best['seq_len'])
loader  = DataLoader(dataset, batch_size=best['batch_size'], shuffle=True)

# Instanciar modelo final
model     = GRUForecast(1, best['hidden_size'], best['num_layers'], best['dropout'])
optimizer = optim.Adam(model.parameters(), lr=best['lr'])
criterion = nn.MSELoss()

# Entrenar final (ej: 30 épocas)
for epoch in range(30):
    model.train()
    for x_batch, y_batch in loader:
        optimizer.zero_grad()
        y_pred = model(x_batch)
        loss   = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

# Pronóstico iterativo para 2022
preds = []
for bar, grp in df_train.groupby('id_bar'):
    vals = grp.sort_values('date')['dengue'].values.tolist()
    seq  = vals[-best['seq_len']:]
    for week in range(1, 53):
        x_input = torch.tensor(seq[-best['seq_len']:], dtype=torch.float32).unsqueeze(0).unsqueeze(-1)
        with torch.no_grad():
            y_hat = model(x_input).item()
        preds.append({
            'id': f"{bar}_2022_{str(week).zfill(2)}",
            'dengue': round(y_hat, 2)
        })
        seq.append(y_hat)

# Crear DataFrame y guardar
df_preds = pd.DataFrame(preds)
fecha_actual = datetime.now().strftime('%Y%m%d_%H%M%S')
df_preds.to_csv(f'sample_submission_gru_{fecha_actual}.csv', index=False)
print('Pronóstico para 2022 guardado en sample_submission.csv')

Pronóstico para 2022 guardado en sample_submission.csv
